<a href="https://colab.research.google.com/github/UW-ERSL/TOuNN/blob/main/TOuNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # run this first time to clone the directory 
# !git clone https://github.com/UW-ERSL/TOuNN.git
# %cd TOuNN/

### Imports

In [ ]:
import numpy as np
import time
# from TOuNNOptimizer import TopologyOptimizer
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.optim as optim
from os import path

from tounn.FE import FE
from tounn.plotUtil import Plotter
from tounn.network import TopNet

import matplotlib.pyplot as plt
from torch.autograd import grad

### Mesh 

In [ ]:
nelx = 121 #61; 
nely = 61 #31; 
elemSize = np.array([1.0, 1.0]);
mesh = {'nelx':nelx, 'nely':nely, 'elemSize':elemSize};

### Material

In [ ]:
matProp = {'E':1.0, 'nu':0.3}; # Structural
matProp['penal'] = 3.; # SIMP penalization constant, starting value

### Boundary Condition

In [ ]:
exampleName = 'TipCantilever'
physics = 'Structural'
ndof = 2*(nelx+1)*(nely+1);
force = np.zeros((ndof,1))
dofs=np.arange(ndof);
fixed = dofs[0:2*(nely+1):1];
force[2*(nelx+1)*(nely+1)-2*nely+1, 0 ] = -1;
symXAxis = {'isOn':False, 'midPt':0.5*nely};
symYAxis = {'isOn':False, 'midPt':0.5*nelx};
bc = {'exampleName':exampleName, 'physics':physics, \
      'force':force, 'fixed':fixed, 'symXAxis':symXAxis, 'symYAxis':symYAxis };

# For more BCs see examples.py

### NN Settings

In [ ]:
nnSettings = {'numLayers':3, 'numNeuronsPerLyr':20}

### Constraints and Projections

In [ ]:
densityProjection = {'isOn':True, 'sharpness':8};
desiredVolumeFraction = 0.3;

### Optimizer settings

In [ ]:
minEpochs = 150; # minimum number of iterations
maxEpochs = 5000; # Max number of iterations

In [ ]:
class TopologyOptimizer:
    def __init__(self, mesh, matProp, bc, nnSettings, \
                  desiredVolumeFraction, densityProjection, overrideGPU = True):

        self.exampleName = bc['exampleName'];
        self.device = self.setDevice(overrideGPU);
        self.boundaryResolution  = 3; # default value for plotting and interpreting
        self.FE = FE(mesh, matProp, bc);
        self.xy = torch.tensor(self.FE.elemCenters, requires_grad = True).\
                                        float().view(-1,2).to(self.device);
        self.xyPlot = torch.tensor(self.FE.generatePoints(self.boundaryResolution, True),\
                        requires_grad = True).float().view(-1,2).to(self.device);
        self.Pltr = Plotter();

        self.desiredVolumeFraction = desiredVolumeFraction;
        self.density = self.desiredVolumeFraction*np.ones((self.FE.numElems));
        self.symXAxis = bc['symXAxis'];
        self.symYAxis = bc['symYAxis'];

        self.densityProjection = densityProjection;

        inputDim = 2; # x and y coordn
        self.topNet = TopNet(nnSettings, inputDim).to(self.device);
        self.objective = 0.;

    def setDevice(self, overrideGPU):
        if(torch.cuda.is_available() and (overrideGPU == False) ):
            device = torch.device("cuda:0");
            print("GPU enabled")
        else:
            device = torch.device("cpu")
            print("Running on CPU")
        return device;

    def applySymmetry(self, x):
        if(self.symYAxis['isOn']):
            xv =( self.symYAxis['midPt'] + torch.abs( x[:,0] - self.symYAxis['midPt']));
        else:
            xv = x[:,0];
        if(self.symXAxis['isOn']):
            yv = (self.symXAxis['midPt'] + torch.abs( x[:,1] - self.symXAxis['midPt'])) ;
        else:
            yv = x[:,1];
        x = torch.transpose(torch.stack((xv,yv)),0,1);
        return x;

    def projectDensity(self, x):
        if(self.densityProjection['isOn']):
            b = self.densityProjection['sharpness']
            nmr = np.tanh(0.5*b) + torch.tanh(b*(x-0.5));
            x = 0.5*nmr/np.tanh(0.5*b);
        return x;

In [ ]:
from torch import nn
class pixelModel(nn.Module):
    def __init__(self, nely=nely, nelx=nelx):
        super().__init__()
        self.nelx = nelx
        self.nely = nely
        self.param = nn.Parameter(torch.zeros(size=(nely, nelx), requires_grad=True))
        
    def forward(self):
        a = self.param.sigmoid()
        return a

In [ ]:
import torchvision
gauss = torchvision.transforms.GaussianBlur(3, sigma=(0.5, .5))

In [ ]:
# plt.imshow(rho_np.reshape(nely, nelx))

In [ ]:
model = pixelModel()

matProp = {'E':1.0, 'nu':0.3}; # Structural
matProp['penal'] = 1.5; # SIMP penalization constant, starting value

plt.close('all');
overrideGPU = False
start = time.perf_counter()
topOpt = TopologyOptimizer(mesh, matProp, bc, nnSettings, \
                  desiredVolumeFraction, densityProjection, overrideGPU);

topOpt.convergenceHistory = {'compliance':[], 'vol':[], 'grayElems':[]};
learningRate = 1e-1;
alphaMax = 100*topOpt.desiredVolumeFraction;
alphaIncrement = 0.05;
# alpha = alphaIncrement; # start
alpha = 0; # start
nrmThreshold = 0.05; # for gradient clipping
# topOpt.optimizer = optim.Adam(topOpt.topNet.parameters(), amsgrad=True, lr=learningRate);
topOpt.optimizer = optim.Adam(model.parameters(), amsgrad=True, lr=learningRate);


topOpt.obj0 = 385.11016982915766
# if(epoch == 0):
#     topOpt.obj0 = ( topOpt.FE.Emax*(rho_np**topOpt.FE.penal)*Jelem).sum()
# For sensitivity analysis, exponentiate by 2p here and divide by p in the loss func hence getting -ve sign

# nn_rho = torch.nn.Parameter(torch.zeros(size=(nelx, nely), requires_grad=True)).flatten()

for epoch in range(maxEpochs):
    topOpt.optimizer.zero_grad();

    # Parameters -> rho
    # x = topOpt.applySymmetry(topOpt.xy);
    # nn_rho = torch.flatten(topOpt.topNet(x)).to(topOpt.device);
    # nn_rho = topOpt.projectDensity(nn_rho);
    nn_rho = model().flatten()
    # print(nn_rho.shape)

    # Do the FEM-related computing
    rho_np = nn_rho.cpu().detach().numpy(); # move tensor to numpy array   
    topOpt.density = rho_np;
    u, Jelem = topOpt.FE.solve(rho_np); # Call FE 88 line code [Niels Aage 2013]
    
    Jelem = np.array(topOpt.FE.Emax*(rho_np**(2*topOpt.FE.penal))*Jelem).reshape(-1);
    Jelem = torch.tensor(Jelem).view(-1).float().to(topOpt.device);
    objective = torch.sum( Jelem / (nn_rho**topOpt.FE.penal))/topOpt.obj0; # compliance

    volConstraint = ((torch.mean(nn_rho)/topOpt.desiredVolumeFraction) - 1.0); # global vol constraint
    currentVolumeFraction = np.average(rho_np);
    topOpt.objective = objective;
    loss = topOpt.objective + alpha * torch.pow(volConstraint, 2);

    alpha = min(alphaMax, alpha + alphaIncrement);
    loss.backward(retain_graph=True);
    # torch.nn.utils.clip_grad_norm_(topOpt.topNet.parameters(), nrmThreshold)
    topOpt.optimizer.step();

    greyElements = sum(1 for rho in rho_np if ((rho > 0.05) & (rho < 0.95)));
    relGreyElements = topOpt.desiredVolumeFraction * greyElements / len(rho_np);
    topOpt.convergenceHistory['compliance'].append(topOpt.objective.item());
    topOpt.convergenceHistory['vol'].append(currentVolumeFraction);
    topOpt.convergenceHistory['grayElems'].append(relGreyElements);
    # topOpt.FE.penal = min(8.0,topOpt.FE.penal + 0.02); # continuation scheme

    if(epoch % 20 == 0):
        titleStr = "Iter {:d} , Obj {:.2F} , vol {:.2F}".format(epoch, topOpt.objective.item()*topOpt.obj0, currentVolumeFraction);
        topOpt.Pltr.plotDensity(topOpt.xy.detach().cpu().numpy(), rho_np.reshape((topOpt.FE.nelx, topOpt.FE.nely)), titleStr);
        print(titleStr);
    if ((epoch > minEpochs ) & (relGreyElements < 0.01)):
        break;

# topOpt.Pltr.plotDensity(topOpt.xy.detach().cpu().numpy(), rho_np.reshape((topOpt.FE.nelx, topOpt.FE.nely)), titleStr);
# print("{:3d} J: {:.2F}; Vf: {:.3F}; loss: {:.3F}; relGreyElems: {:.3F} "\
#         .format(epoch, topOpt.objective.item()*topOpt.obj0, currentVolumeFraction, loss.item(), relGreyElements));

# print("Final J : {:.3f}".format(topOpt.objective.item()*topOpt.obj0));
# topOpt.Pltr.plotConvergence(topOpt.convergenceHistory);

# x = topOpt.applySymmetry(topOpt.xyPlot);
# rho = torch.flatten(topOpt.projectDensity(topOpt.topNet(x)));
# rho_np = rho.cpu().detach().numpy();

# titleStr = "Iter {:d} , Obj {:.2F} , vol {:.2F}".format(epoch, topOpt.objective.item()*topOpt.obj0, currentVolumeFraction);
# topOpt.Pltr.plotDensity(topOpt.xyPlot.detach().cpu().numpy(), rho_np.reshape((topOpt.FE.nelx*topOpt.boundaryResolution, topOpt.FE.nely*topOpt.boundaryResolution)), titleStr);

In [ ]:
# x = topOpt.applySymmetry(topOpt.xy);
# nn_rho = torch.flatten(topOpt.topNet(x)).to(topOpt.device);
# nn_rho = topOpt.projectDensity(nn_rho);

# # Do the FEM-related computing
# rho_np = nn_rho.cpu().detach().numpy(); # move tensor to numpy array   
# plt.imshow(rho_np.reshape((topOpt.FE.nelx, topOpt.FE.nely)).T)
# plt.colorbar()